In [ ]:
# Parameters
day_obs = '2024-06-26'
instruments = 'latiss, lsstcomcamsim, lsstcomcam'

In [ ]:
import os
import pandas as pd
import matplotlib.pylab as plt
from IPython.display import display, Markdown, display_markdown

from lsst.summit.utils import ConsDbClient

In [ ]:
URL = "http://consdb-pq.consdb:8080/consdb" # Don't use this one
os.environ["no_proxy"] += ",.consdb"
%matplotlib inline
day_obs_int = int(day_obs.replace('-', ''))

In [ ]:
client = ConsDbClient(URL)
display_markdown('### Consolidated Database is accessible',raw=True)

In [ ]:

psf_query = f'''
    SELECT psf_area FROM cdb_lsstcomcamsim.ccdvisit1_quicklook
    where day_obs = {day_obs_int}
'''

sky_brightness_query = f'''
'''

zero_point_query = f'''
    SELECT zero_point FROM cdb_lsstcomcamsim.ccdvisit1_quicklook
    where day_obs = {day_obs_int}
'''

ra_dec_query = f'''
    SELECT s_ra, s_dec FROM cdb_lsstcomcamsim.ccdvisit1_quicklook
    where day_obs = {day_obs_int}
'''

plt.style.use('seaborn-v0_8-bright')
def plot(y, x):
    # plot
    fig, ax = plt.subplots()
    ax.scatter(x, y)#, linewidth=2.0, color="#058b8c")

    plt.show()


In [ ]:
day_obs_int = int(day_obs.replace('-', ''))

instrument_list = instruments.split(',',' ')

for instrument in instrument_list:
    ccdvisit1_quicklook = f'''
        SELECT * FROM cdb_{instrument}.ccdvisit1_quicklook
    '''

    visit1 = f'''
        SELECT * FROM cdb_{instrument}.visit1
    '''

    # Join Visit1 and ccdVisit1 to access data and day obs
    visits = client.query(visit1).to_pandas()
    quicklook = client.query(ccdvisit1_quicklook).to_pandas()

    visits = visits.join(quicklook, on='visit_id',lsuffix='',rsuffix='_q')

    # If we see data exist in psf, zero, then we should pare down like visits_today below
    visits_w_psf = visits[visits['psf_area'].notna()]
    time = visits_w_psf['obs_start']
    print(f'length of visits with psf_area populated {len(visits_w_psf)}')
    display_markdown('# psf_area vs obs_start', raw=True)
    plot(time, visits_w_psf['psf_area'])

    visits_w_zero = visits[visits['zero_point'].notna()]
    time = visits_w_zero['obs_start']
    print(f'length of visits with zero_point populated {len(visits_w_zero)}')
    display_markdown('# zero_point vs obs_start', raw=True)
    plot(time, visits_w_zero['zero_point'])

    # Pare down to only day obs
    visits_today = visits[(visits['day_obs'] == day_obs_int)]

    ra = visits_today['s_ra']
    dec = visits_today['s_dec']
    display_markdown('# Ra Dec proof they populated', raw=True)
    plot(ra, dec)